In [3]:
# Imports
import os 
import torch
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import ModelCheckpoint
import torchvision
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sym import LCN, ProgressBar, Classification_report, Logger
device = "cuda:0" if torch.cuda.is_available() else "cpu"
plt.style.use("dark_paper")

/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
testset = torch.load("./data/testset.pt")
trainset = torch.load("./data/trainset.pt")

In [5]:
# Aggregate validation losses -> DONE!
# Change back to SGD -> DONE!
# Shit on computaional separation paper
# Reporoduce their experiments
# Show that the bounds are consistent
# By Friday have 5 pages
# Fix LCN Code
# Think about better problem statement

In [35]:
class Model(pl.LightningModule):
    def __init__(self, model, nfilters = 32):
        super().__init__()
        self.model = model(nfilters=nfilters)
        self.hparams["Params"] = sum([x.size().numel() for x in self.model.parameters()])
        self.save_hyperparameters({"nfilters" : nfilters, "Model" : self.model.__repr__().replace("\n", "")})
    def forward(self, x):
        return self.model(x)
    def training_step(self, batch, batch_idx):
        x, y = batch
        yhat = self(x)
        loss = self.Loss(yhat, y)
        self.log('train_loss', loss)
        return loss
    def validation_step(self, batch, batch_idx):
        x, y = batch
        yhat = self(x)
        loss = self.Loss(yhat, y)
        preds = torch.argmax(yhat, dim=1)
        acc = accuracy(preds, y)
        # Calling self.log will surface up scalars for you in TensorBoard
        metrics = {'val_loss': loss, 'val_acc': acc}
        self.log_dict(metrics, prog_bar=True,logger=True,on_epoch=True,on_step=False)
        self.logger.log_hyperparams(self.hparams,metrics=metrics)
        return metrics
    def validation_epoch_end(self, outputs):
        val_loss_mean = 0
        val_acc_mean = 0
        for output in outputs:
            val_loss_mean += output['val_loss']
            val_acc_mean += output['val_acc']
        val_loss_mean /= len(outputs)
        val_acc_mean /= len(outputs)
        metrics = {'val_loss': val_loss_mean, 'val_acc': val_acc_mean}
#         self.log_dict(metrics, prog_bar=True,logger=True,on_epoch=True,on_step=False)
#         self.logger.log_hyperparams(self.hparams,metrics=metrics)
        return
    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)
    def configure_optimizers(self, learning_rate=1e-1):
        optimizer = torch.optim.SGD(self.model.parameters(), lr=learning_rate)
        return optimizer
    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == 'fit' or stage is None:
            self.mnist_train, self.mnist_val = torch.utils.data.random_split(trainset, [
                                                                             50000, 10000])
        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.mnist_test = testset
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_train, num_workers=6, batch_size=320)
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_val, num_workers=6, batch_size=320,)
    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_test, num_workers=6, batch_size=320)
    def Loss(self,preds,targets):
        preds = torch.nn.functional.softmax(preds,dim=-1)
        targets = torch.nn.functional.one_hot(targets,10)
        loss = ((preds-targets)**2).mean()
#         loss = torch.nn.CrossEntropyLoss()(preds,targets)
        return loss 

/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
class SFCN(torch.nn.Module):
    def __init__(self,input_dim=280,nfilters=32,pool_size=100,C=10):
        super().__init__()
        self.nfilters = nfilters
        self.pool_size = pool_size
        self.fc1 = torch.nn.Linear(input_dim**2, nfilters*pool_size)
        self.pool = torch.nn.MaxPool1d(pool_size,1)
        self.out = torch.nn.Linear(nfilters, C)
    def forward(self,x):
        x = x.flatten(start_dim=1)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.pool(x.view((-1,self.nfilters,self.pool_size)))
        return self.out(x.view(-1,self.nfilters))

In [37]:
class CNN(torch.nn.Module):
    def __init__(self,nfilters=32):
        super().__init__()
        self.nfilters = nfilters
        self.model = torch.nn.Sequential(torch.nn.Conv2d(1, nfilters, kernel_size=28, stride=28),
                          #torch.nn.BatchNorm2d(nfilters),
                          torch.nn.ReLU(),
                          torch.nn.MaxPool2d(10, 1),
                          torch.nn.Flatten(),
                          torch.nn.Linear(nfilters, 10))
    def forward(self,x):
        return self.model(x)

In [38]:
logdir = "./lightning_logs/MSE"
os.mkdir(logdir)
epochs = 10

In [39]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(10)
    bar = ProgressBar()
    log = Logger(logdir, name="CNN",default_hp_metric=False)
    path = logdir+f"/CNN/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    cnn = Model(CNN,nfilters)
    trainer.fit(cnn)

Missing logger folder: ./lightning_logs/MSE/CNN
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 25.4 K


Epoch 9: 100%|##########| 189/189 [00:08<00:00, 23.50it/s, loss=0.009, v_num=0, val_loss=0.0124, val_acc=0.931]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 12.7 K



Epoch 9: 100%|##########| 189/189 [00:07<00:00, 23.79it/s, loss=0.013, v_num=1, val_loss=0.0166, val_acc=0.901]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 6.4 K 



Epoch 9: 100%|##########| 189/189 [00:08<00:00, 22.99it/s, loss=0.058, v_num=2, val_loss=0.0595, val_acc=0.452]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 3.2 K 



Epoch 9: 100%|##########| 189/189 [00:08<00:00, 23.36it/s, loss=0.082, v_num=3, val_loss=0.0822, val_acc=0.204]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 1.6 K 



Epoch 9: 100%|##########| 189/189 [00:08<00:00, 21.64it/s, loss=0.091, v_num=4, val_loss=0.0905, val_acc=0.0968]


In [40]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(42)
    bar = ProgressBar()
    log = Logger(logdir, name="LCN",default_hp_metric=False)
    path = logdir+f"/LNN/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    lcn = Model(LCN,nfilters)
    trainer.fit(lcn)

Missing logger folder: ./lightning_logs/MSE/LCN
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | LCN  | 2.5 M 


Epoch 9: 100%|##########| 189/189 [00:14<00:00, 12.61it/s, loss=0.181, v_num=0, val_loss=0.18, val_acc=0.0996]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | LCN  | 1.3 M 



Epoch 9: 100%|##########| 189/189 [00:11<00:00, 16.28it/s, loss=0.173, v_num=1, val_loss=0.173, val_acc=0.134]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | LCN  | 636 K 



Epoch 9: 100%|##########| 189/189 [00:09<00:00, 19.37it/s, loss=0.170, v_num=2, val_loss=0.172, val_acc=0.137]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | LCN  | 318 K 



Epoch 9: 100%|##########| 189/189 [00:08<00:00, 21.20it/s, loss=0.161, v_num=3, val_loss=0.16, val_acc=0.14]  

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | LCN  | 159 K 



Epoch 9: 100%|##########| 189/189 [00:08<00:00, 22.06it/s, loss=0.142, v_num=4, val_loss=0.142, val_acc=0.123]


In [42]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(42)
    bar = ProgressBar()
    log = Logger(logdir, name="FCN",default_hp_metric=False)
    path = logdir+f"/FCN/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    fcn = Model(SFCN,nfilters)
    trainer.fit(fcn)

# Cross Entropy 

In [8]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(42)
    bar = ProgressBar()
    log = Logger(logdir, name="CNN",default_hp_metric=False)
    path = logdir+f"/CNN/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    cnn = Model(CNN,nfilters)
    trainer.fit(cnn)

Missing logger folder: ./lightning_logs/CNN
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | CNN              | 25.4 K
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.82it/s, loss=0.710, v_num=0, val_loss=0.716, val_acc=0.79] 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | CNN              | 12.7 K
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.96it/s, loss=1.767, v_num=1, val_loss=1.76, val_acc=0.255]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | CNN              | 6.4 K 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.91it/s, loss=2.301, v_num=2, val_loss=2.3, val_acc=0.115]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | CNN              | 3.2 K 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.47it/s, loss=2.302, v_num=3, val_loss=2.3, val_acc=0.111]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | CNN              | 1.6 K 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.82it/s, loss=2.302, v_num=4, val_loss=2.3, val_acc=0.111]


In [9]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(42)
    bar = ProgressBar()
    log = Logger(logdir, name="LCN",default_hp_metric=False)
    path = logdir+f"/LNN/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    lcn = Model(LCN,nfilters)
    trainer.fit(lcn)

Missing logger folder: ./lightning_logs/LCN
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | LCN              | 2.5 M 
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:15<00:00, 12.19it/s, loss=1.546, v_num=0, val_loss=2.05, val_acc=0.432]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | LCN              | 1.3 M 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:12<00:00, 15.60it/s, loss=1.831, v_num=1, val_loss=2.25, val_acc=0.319]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | LCN              | 636 K 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:10<00:00, 18.52it/s, loss=2.032, v_num=2, val_loss=2.09, val_acc=0.217]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | LCN              | 318 K 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:09<00:00, 20.32it/s, loss=2.159, v_num=3, val_loss=2.26, val_acc=0.157]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | LCN              | 159 K 
1 | Loss  | CrossEntropyLoss | 0     



Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.35it/s, loss=2.213, v_num=4, val_loss=2.29, val_acc=0.136]


In [10]:
for nfilters in 2**np.arange(5,0,-1):
    pl.seed_everything(42)
    bar = ProgressBar()
    log = Logger(logdir, name="FCN",default_hp_metric=False)
    path = logdir+f"/FCN/version_{log.version}"
    checkpoint_callback = ModelCheckpoint(dirpath=path,filename="weights.cpkt",save_top_k=1,monitor='val_loss',mode='min')
    trainer = pl.Trainer(logger=log, gpus=1, max_epochs=epochs, progress_bar_refresh_rate=1,callbacks=[bar,checkpoint_callback])
    fcn = Model(SFCN,nfilters)
    trainer.fit(fcn)

Missing logger folder: ./lightning_logs/FCN
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | SFCN             | 250 M 
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:18<00:00,  9.91it/s, loss=8.814, v_num=0, val_loss=28.9, val_acc=0.904] 


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | SFCN             | 125 M 
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:13<00:00, 13.44it/s, loss=1.997, v_num=1, val_loss=5.54, val_acc=0.895] 


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | SFCN             | 62.7 M
1 | Loss  | CrossEntropyLoss | 0     


...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | SFCN             | 31.4 M
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:10<00:00, 18.20it/s, loss=0.411, v_num=3, val_loss=0.798, val_acc=0.826]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name  | Type             | Params
-------------------------------------------
0 | model | SFCN             | 15.7 M
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:10<00:00, 18.51it/s, loss=1.856, v_num=4, val_loss=1.88, val_acc=0.274]
